In [ ]:
import numpy as np
import numpy.random as nr
import sympy as sy
import IPython.display as disp

sy.init_printing()



# 선형 연립 방정식<br>Systems of Linear Equations



미지수가 3개인 선형 연립 방정식을 생각해 보자.<br>Let's think about a system of linear equations with three unknowns.



In [ ]:
n = 3

x = np.array(sy.symbols(f'x:{n}'))
x



세 미지수를 모두 결정하려면, 보통 세개의 서로 선형 독립인 방정식이 필요하다.<br>To decide all three unknowns, usually we need three linearly independent equations.



In [ ]:
a = np.array(sy.symbols(
    f'a:{n}(:{n})'
)).reshape((n, n)).tolist()

b = sy.symbols(f'b:{n}')



In [ ]:
eqs = []
for coefs, const in zip(a, b):
    lhs = sum([aij * xj for aij, xj in zip(coefs, x)])
    eq = sy.Eq(lhs, const)
    eqs.append(eq)
    disp.display(eq)



행렬 형태로 정리해 보자<br>Let's rewrite in the matrix form



In [ ]:
matA = sy.Matrix(a)
vecB = sy.Matrix(b)
vecX = sy.Matrix(x)

eq_mat = sy.Eq(matA * vecX, vecB)

eq_mat



여기서 계수 행렬과 상수 벡터만 생각해 보자.<br>Here, let's just think about the coefficient matrix and constant vector.



In [ ]:
matAb = matA.col_insert(n, vecB)
matAb



## 가우스 소거법<br>Gauss Elimination



다음 비디오는 가우스 소거법을 소개한다.<br>
The following video introduces the Gauss Elimiation. (00:17 ~ 19:30)

[![MIT OCW 18.06 Lecture 2 Gauss Elimination](https://i.ytimg.com/vi/QVKj3LADCnA/hqdefault.jpg)](https://www.youtube.com/watch?v=QVKj3LADCnA&list=PLE7DDD91010BC51F8&index=3&start=18&end=1169)



임의의 행렬을 생각해 보자.<br>Let's think about a matrix of random values.



In [ ]:
A = nr.randint(-100, 100, size=(n, n)) * 1.0
b = nr.randint(-100, 100, size=(n, 1)) * 1.0
Ab = np.hstack((A, b))
Ab



우선 첫 행의 첫 열 원소에 pivot 이라는 이름을 준다.<br>First, let's designate the first element of the first row as pivot.



In [ ]:
p = 0
pivot = Ab[p, p]



두번째 행 첫 열 원소를 pivot 으로 나눈 비를 계산한다.<br>
Divide the element at the first column of the second row with pivot



In [ ]:
i = p + 1
ratio = Ab[i, p] / pivot
ratio



첫 행에 이 비를 곱한 후 둘째 행에서 뺀다.<br>Multiply the first row with this ratio and subtract from the second row.



In [ ]:
Ab[i, :]  = Ab[i, :] - ratio * Ab[p, :]
Ab



세번째 행에 대해서도 반복한다.<br>
Let's repeat for the third row.<br>
세번째 행 첫 열 원소를 pivot 으로 나눈 비를 계산한다.<br>
Divide the element at the first column of the third row with pivot.



In [ ]:
i += 1
ratio = Ab[i, p] / pivot
ratio



첫 행에 이 비를 곱한 후 세째 행에서 뺀다.<br>Multiply the first row with this ratio and subtract from the third row.



In [ ]:
Ab[i, :]  = Ab[i, :] - ratio * Ab[p, :]
Ab



이제 p 에 1을 더하고 반복하자.<br>Now let's add 1 to `p` and repeat.



In [ ]:
p += 1
pivot = Ab[p, p]



`p+1` 행 `p` 열 원소를 `pivot` 으로 나눈 비를 계산한다.<br>
Divide the element at the `p`th column of the `p+1`th row with pivot



In [ ]:
i = p + 1
ratio = Ab[i, p] / pivot
ratio



`p` 행에 이 비를 곱한 후 `p+1` 행에서 뺀다.<br>
Multiply the `p`th row with this ratio and subtract from the `p+1`th row.



In [ ]:
Ab[i, :]  = Ab[i, :] - ratio * Ab[p, :]
Ab



이런 식으로 왼쪽 위로부터 오른쪽 아래로의 주대각선 아래 원소를 모두 0으로 만든다.<br>This way, make all elements below main diagonal, from the left upper corner to the right lower direction, zero.



## 후진대입법<br>Backward substitution



주대각선 아래가 모두 0이라면 아래와 같이 생각해 볼 수 있다.<br>If all elements below the main diagonal are zeros, we may think as follows.



In [ ]:
alpha = np.array(sy.symbols(
    f'alpha:{n}(:{n})'
)).reshape((n, n)).tolist()

beta = sy.symbols(f'beta:{n}')



In [ ]:
eqs2 = []
for p in range(n):
    
    lhs_list = []
    for i in range(p, n):
        lhs_list.append(alpha[p][i]*x[i])

    eq = sy.Eq(sum(lhs_list), beta[p])
    eqs2.append(eq)

for eq in eqs2:
    disp.display(eq)



맨 마지막 행에서 마지막 미지수를 알 수 있다.<br>
From the last row, we can find the last unknown.



In [ ]:
sol = sy.Matrix([None] * n)



In [ ]:
sol_n_1 = sy.solve(eqs2[-1], x[-1])
sol[-1] = sol_n_1[0]
disp.display(sol)



그 하나 앞 미지수는 마지막에서 두번째 방정식에서 구할 수 있다.<br>We can find the second last unknown from the second last equation.



In [ ]:
eqs2[-2].subs(x[-1], sol[-1])



In [ ]:
sol_n_2 = sy.solve(eqs2[-2].subs(x[-1], sol[-1]), x[-2])
sol[-2] = sol_n_2[0]
disp.display(sol)



반복하면 모든 해를 구할 수 있다.<br>We can find all solutions this way.



`numpy.linalg` 의 `solve()` 함수를 이용할 수도 있다.<br>We can use `solve()` of `numpy.linalg`.



In [ ]:
import numpy.linalg as nl

x_sol = nl.solve(A, b)
x_sol



In [ ]:
assert 1e-7 > nl.norm(np.matrix(A) * np.matrix(x_sol) - b)



## 연습 문제<br>Exercise



위 방법을 적용 가능한 공학 문제 사례를 설명하고 `numpy.linalg.solve()`로 해를 구해 보시오. 이렇게 구한 해가 맞는지 어떻게 확인할 수 있는가?<br>
Describe an engineering problem that we can apply the method above and find the solution using `numpy.linalg.solve()`. How can we verify if the solution is correct?



## 참고문헌<br>References



* Gilbert Strang. 18.06 Linear Algebra. Spring 2010. Massachusetts Institute of Technology: MIT OpenCourseWare, https://ocw.mit.edu. License: Creative Commons BY-NC-SA.

## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

